In [ ]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

# !pip install fenics # for local machine

In [ ]:
import json
import os
import time
from dolfin import *
# from fenics import * # for local machine
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function

# Evaluation Points & Solutions

In [ ]:
# Load evaluation points
with open('2D_Transient_Heat_eval_points.json', 'r') as f:
    data_points = json.load(f)

# Evaluation coordinates & time
mesh_coords = data_points["mesh_coord"]["0"]
dt_coords = data_points["dt_coord"]["0"] # [[0.0], [0.1], ..., [1.0]]
times = [dt_coord[0] for dt_coord in dt_coords]  # unpack to [0.0, 0.1, ...]

# Number of time intervals
nt_steps = len(times) - 1

# Number to divide in both spatial dimensions
ns_list = [5, 10, 30, 50]

# Load evaluation solutions (by Ground Truth FEM)
with open('2D_Transient_Heat_eval_solutions.json', 'r') as f:
    data_sol = json.load(f)

# Ground truth soution (100 x 100 cells)
u_true = np.array(data_sol)

# Boundary Conditions
Detailed explanation of FEM implementation is available in "2D_Transient_Heat_Ground_Truth.py".



In [ ]:
def boundary(x, on_boundary):
    return on_boundary

# Initial Condition

In [ ]:
# u0 = exp(-50*((x-0.5)^2 + (y-0.5)^2))
class InitialCondition(UserExpression):
    def eval(self, values, x):
        values[0] = np.exp(-50.0 * ((x[0] - 0.5)**2 + (x[1] - 0.5)**2))
    def value_shape(self):
        return ()

# FEM Approximate Solution

In [ ]:
# Containers for the results
y_results, times_solve, times_eval, l2_rel\
    = dict({}), dict({}), dict({}), dict({})

count = 0
for ns in ns_list:
    time_solving = 0.
    time_evaluation = 0.
    for i in range(0, 10): # Loop over 10 solving runs
        # Mesh
        mesh = UnitSquareMesh(ns, ns)

        # Function space
        V = FunctionSpace(mesh, "P", 1)

        # Dirichlet BC
        bc = DirichletBC(V, Constant(0.0), boundary) # u = "0" at x = "boundary"

        # Initial condition
        u_n = Function(V) # u_n : solution at time step n
        u_n.interpolate(InitialCondition()) # assign u_0(x,y) to u_n at t=0

        # Initialize time & time step
        t = Constant(0.0)
        dt = Constant(0.0)

        # Source term
        # f(x,y,t) = 10*sin(pi*x)*sin(pi*y)*cos(2*pi*t)
        f = Expression("10.0*sin(pi*x[0])*sin(pi*x[1])*cos(2.0*pi*t)", degree = 2, t=t)

        # Trial and test functions
        u = Function(V) # shouldn't it be TrialFunction(V)?
        v = TestFunction(V)

        # Weak form
        # R = (u - u_n)*v + dt*(grad(u), grad(v)) = dt*f_expr*v
        R = (u-u_n)*v*dx + dt*dot(grad(u), grad(v))*dx - dt*f*v*dx

        # Jacobian
        jac = derivative(R, u) # dF/du(jacobian)

        # Save t=0 solution (pvd)
        save_dir = './vtu' + '/mesh_' + str(ns)
        os.makedirs(save_dir, exist_ok = True)

        if i == 9: # only the last run is stored
            File(f"{save_dir}/solution_000.pvd") << u_n

        # Save t=0 solution (json)
        sol_list = []
        sol0 = []
        for (x, y) in mesh_coords:
            u_0 = u_n(Point(x, y))
            sol0.append(u_0)
        sol_list.append(sol0)

        # Time-stepping loop
        for n in range(nt_steps):
            t_prev = times[n] # t_(n-1)
            t_curr = times[n+1] # t_n

            # Update dt
            dt.assign(t_curr - t_prev)

            # Update t (source term is also automatically updated w/ this line)
            t.assign(t_curr)

            # Solve
            start_time = time.time()
            solve(R == 0, u, bc, J=jac)
            solve_time = time.time()
            time_solving += solve_time - start_time

            # Evaluate & save in the container
            start_time2 = time.time()
            sol = []
            for (x, y) in mesh_coords:
                sol.append(u(Point(x, y)))
            eval_time = time.time()
            sol_list.append(sol)
            time_evaluation += eval_time - start_time2

            # Assign new solution(u) into previous solution(u_n) for the next step
            u_n.assign(u)

            # Save solution (pvd)
            if i == 9: # only the last run is stored
                if (n + 1) % 1 == 0: # "% 10" is for every 10th step
                    print(f"Time step {n + 1}, t = {t_curr:.4f}")
                    File(f"{save_dir}/solution_{n + 1:03d}.pvd") << u

        u_approx = np.array(sol_list)

    # L2 error
    l2 = np.linalg.norm(u_approx - u_true)
    l2_rel_single = l2 / np.linalg.norm(u_true)

    print(f'MESH {ns} x {ns}, RUN {i}')
    print('Average Solution Time : ', time_solving / 10)
    print('Average Evaluation Time : ', time_evaluation / 10)
    print('Average Accuracy : ', l2_rel_single)

    y_results[count] = u_approx.tolist()
    times_solve[count] = time_solving / 10
    times_eval[count] = time_evaluation / 10
    l2_rel[count] = l2_rel_single
    count += 1

    results = dict({'y_results': y_results,
                    'times_solve': times_solve,
                    'times_eval': times_eval,
                    'l2_rel': l2_rel,
                    'mesh_nums': ns_list})

# Save solution (json)
sol_json = "FEM_results.json"
with open(sol_json, 'w') as f:
    json.dump(results, f)

print(f"Solution created: {sol_json}")

# Contour Plot (Solution)

In [ ]:
# Load evaluation points
with open('2D_Transient_Heat_eval_points.json', 'r') as f:
    data_points = json.load(f)

# Evaluation coordinates & time
mesh_coords = data_points["mesh_coord"]["0"] # list
dt_coords = data_points["dt_coord"]["0"] # [[0.0], [0.1], ..., [1.0]]
times = [dt_coord[0] for dt_coord in dt_coords]  # unpack to [0.0, 0.1, ...]


# Load evaluation results (by FEM)
with open('FEM_results.json', 'r') as f:
    data_results = json.load(f)

# Slice results data
ns_list = data_results['mesh_nums'] # list
l2_rel = data_results['l2_rel']
times_solve = data_results['times_solve']
times_eval = data_results['times_eval']
y_results = data_results['y_results']

# Indices of dict
idx_str = list(y_results.keys())

# x, y coordinates
mesh_coords = np.array(mesh_coords)
X = mesh_coords[:, 0]
Y = mesh_coords[:, 1]

for idx, ns in zip(idx_str, ns_list):
    # Get the approximate solution for this architecture
    u_approx = np.array(y_results[idx])  # shape: (n_times, n_points)

    # Contour plot settings
    # For a consistent scale across all time steps
    u_min = u_approx.min()
    u_max = u_approx.max()
    # Create n levels between u_min & u_max:
    num_levels = 80
    levels = np.linspace(u_min, u_max, num_levels)

    for t in range(len(times)):

        u_approx_t = u_approx[t, :]

        fig = plt.figure(figsize=(6, 5))
        sc1 = plt.tricontourf(
            X, Y, u_approx_t,
            levels=levels,
            cmap='viridis')
        plt.title(f"FEM Solution (Mesh Numbers={ns}, t_step={t})")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.colorbar(sc1, shrink=0.7)

        plt.tight_layout()

        # Save figures
        fig_dir = f'./fig/sol_contour/FEM/mesh_num_{ns}'
        if not os.path.exists(fig_dir):
            os.makedirs(fig_dir, exist_ok=True)

        filename = os.path.join(fig_dir, f'sol_{t:04d}.png')
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close(fig)

# Plot (L2 Error vs. Mesh Size )

In [ ]:
# Plot Mesh Size-Relative L2 Error
with open('FEM_results.json', 'r') as f:
    data_results = json.load(f)

ns_list = data_results['mesh_nums'] # list
l2_rel = data_results['l2_rel']

# Mesh size
h = [1.0 / ns for ns in ns_list]

# L2 errors
l2_rel_values = [l2_rel[str(i)] for i in range(len(l2_rel))]

# Plot
plt.figure(figsize=(6, 4))
plt.loglog(h, l2_rel_values, marker='o', label='Relative L2 error')
plt.xlabel('Mesh size')
plt.ylabel('Relative L2 error')
plt.title('Relative L2 Error vs. Mesh Size')
plt.grid(True, which="both", ls="--")
plt.legend()

plt.savefig("L2_error.png", dpi=300)
plt.show()

# Convergence ratio
# Compare the ratio of the last two errors => log2(e_j / e_{j+1})
h_rate_log = np.log(h[-2] / h[-1])
l2_rel_rate_log = np.log(l2_rel_values[-2] / l2_rel_values[-1])
convergence_rate = l2_rel_rate_log / h_rate_log
print("Convergence ratio : ", convergence_rate)